In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-text
import tensorflow_text as text

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import load_model
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.0 MB/s 
     |████████████████████████████████| 588.3 MB 18 kB/s 
     |████████████████████████████████| 439 kB 57.1 MB/s 
     |████████████████████████████████| 6.0 MB 52.4 MB/s 
     |████████████████████████████████| 1.7 MB 46.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

In [ ]:
hotel_reviews = 'hotel_reviews'
movie_reviews = 'movie_reviews'
lambert_trained_on_movie_reviews = 'lambert_trained_on_movie_reviews'
lambert_trained_on_hotel_reviews = 'lambert_trained_on_hotel_reviews'

In [ ]:
models_dict = dict()
models_dict[lambert_trained_on_movie_reviews] = load_model('/content/drive/MyDrive/data_for_colab/lambert_trained_on_imdb_29_december')
models_dict[lambert_trained_on_hotel_reviews] = load_model('/content/drive/MyDrive/data_for_colab/lambert_trained_on_hotel_reviews_29_december')

In [ ]:
path_to_movie_reviews_dataset = '/content/drive/MyDrive/data_for_colab/IMDB Dataset.csv'
path_to_hotel_reviews_dataset = '/content/drive/MyDrive/data_for_colab/tripadvisor_hotel_reviews.csv'

In [ ]:
def test_model_on_dataset_and_write_results_to_dataframe(model_name=None, dataset_name=None, dataframe_to_write_answer=None):

    global hotel_reviews, movie_reviews
    global lambert_trained_on_movie_reviews
    global lambert_trained_on_hotel_reviews
    global models_dict
    global path_to_hotel_reviews_dataset
    global path_to_movie_reviews_dataset

    if model_name is None or dataset_name is None or dataframe_to_write_answer is None:
        raise ValueError("Wrong arguments passed to function: there are none arguments!")
    if model_name != lambert_trained_on_movie_reviews and model_name != lambert_trained_on_hotel_reviews:
        raise ValueError("Wrong model_name!")
    if dataset_name != movie_reviews and dataset_name != hotel_reviews:
        raise ValueError("Wrong dataset_name!")

    model = models_dict[model_name] # загрузили пользователем заданную модель 
    # (она уже обучена на определенном датасете)

    if dataset_name == movie_reviews: # if you edit this line, edit the next line too!
        df = pd.read_csv(path_to_movie_reviews_dataset)
        df['is_positive'] = (df['sentiment'] == 'positive').astype(int)
        df.drop(columns=['sentiment'], inplace=True)
        df.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
        df_positive = df[df['LABEL_COLUMN']==1]
        df_negative = df[df['LABEL_COLUMN']==0]
        # Для тестовой выборки берем последние 10% негативных отзывов и последние 10% позитивных отзывов
        n_test = df_negative.shape[0] // 10  # в оригинале df_negative.shape[0] // 10
        df_negative_test = df_negative.tail(n_test)
        n_test = df_positive.shape[0] // 10  # в оригинале df_positive.shape[0] // 10
        df_positive_test = df_positive.tail(n_test)
        df_balanced_test = pd.concat([df_negative_test, df_positive_test])

    elif dataset_name == hotel_reviews: # if you edit this line, edit the next line too!
        df = pd.read_csv(path_to_hotel_reviews_dataset)
        df = df[df.Rating != 3]
        df['is_positive'] = (df['Rating'] >= 4).astype(int)
        df.drop(columns=['Rating'], inplace=True)
        df.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
        df_positive = df[df['LABEL_COLUMN']==1]
        df_negative = df[df['LABEL_COLUMN']==0]
        # Тестовая выборка
        n_test = (df_negative.shape[0] // 4) * 3  # в оригинале (df_negative.shape[0] // 4) * 3
        df_negative_test = df_negative.tail(n_test)
        n_test = (df_positive.shape[0] // 20) * 3  # в оригинале (df_positive.shape[0] // 20) * 3
        df_positive_test = df_positive.tail(n_test)
        df_balanced_test = pd.concat([df_negative_test, df_positive_test])

    X_test = df_balanced_test['DATA_COLUMN'].squeeze()
    y_test = df_balanced_test['LABEL_COLUMN'].squeeze()

    y_predicted = model.predict(X_test)
    y_predicted = np.where(y_predicted > 0.5, 1, 0)

    row_name_in_dataframe_to_write_answer = model_name + ' tested on ' + dataset_name + ' dataset'

    dataframe_to_write_answer.loc[row_name_in_dataframe_to_write_answer, 'accuracy'] = accuracy_score(y_test, y_predicted)
    dataframe_to_write_answer.loc[row_name_in_dataframe_to_write_answer, 'precision'] = precision_score(y_test, y_predicted)
    dataframe_to_write_answer.loc[row_name_in_dataframe_to_write_answer, 'recall'] = recall_score(y_test, y_predicted)
    dataframe_to_write_answer.loc[row_name_in_dataframe_to_write_answer, 'f1_score'] =  f1_score(y_test, y_predicted)
    print(model_name, ' on dataset', dataset_name, 'result', accuracy_score(y_test, y_predicted), precision_score(y_test, y_predicted), recall_score(y_test, y_predicted), f1_score(y_test, y_predicted) )

In [ ]:
answer_dataframe = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [ ]:
for cur_model_name in [lambert_trained_on_hotel_reviews, lambert_trained_on_movie_reviews]:
    for cur_dataset_name in [hotel_reviews, movie_reviews]:
        test_model_on_dataset_and_write_results_to_dataframe(cur_model_name, cur_dataset_name, answer_dataframe)

146/146 [==============================] - 8013s 55s/step
lambert_trained_on_hotel_reviews  on dataset hotel_reviews result 0.792763862128024 0.8405263157894737 0.7060123784261715 0.7674195098510331
157/157 [==============================] - 8368s 53s/step
lambert_trained_on_hotel_reviews  on dataset movie_reviews result 0.5346 0.7492795389048992 0.104 0.18264840182648404
146/146 [==============================] - 7775s 53s/step
lambert_trained_on_movie_reviews  on dataset hotel_reviews result 0.6745878826803682 0.611881785283474 0.8969938107869142 0.7275008963786304
157/157 [==============================] - 8304s 53s/step
lambert_trained_on_movie_reviews  on dataset movie_reviews result 0.7562 0.7280170879316483 0.818 0.7703899039367113


In [ ]:
answer_dataframe

,accuracy,precision,recall,f1_score
lambert_trained_on_hotel_reviews tested on hotel_reviews dataset,0.792764,0.840526,0.706012,0.76742
lambert_trained_on_hotel_reviews tested on movie_reviews dataset,0.5346,0.74928,0.104,0.182648
lambert_trained_on_movie_reviews tested on hotel_reviews dataset,0.674588,0.611882,0.896994,0.727501
lambert_trained_on_movie_reviews tested on movie_reviews dataset,0.7562,0.728017,0.818,0.77039
